In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time
import os

def download_1min_currency_data(currency_pairs, output_dir="currency_data"):
    """
    Pull 1-minute currency data from Yahoo Finance for the maximum available period (last 7 days).
    
    Parameters:
    -----------
    currency_pairs : list
        List of currency pairs (e.g., ['EURUSD=X', 'GBPUSD=X'])
    output_dir : str, default "currency_data"
        Directory to save the data
        
    Notes:
    ------
    1-minute data is only available for the last 7 days on Yahoo Finance
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Calculate date range (last 7 days)    
    end_date = datetime.now()
    start_date = end_date - timedelta(days=7)
    
    print(f"Downloading 1-minute data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    
    all_data = {}
    
    for pair in currency_pairs:
        try:
            print(f"Downloading {pair} data at 1-minute interval...")
            
            # Download data
            data = yf.download(
                tickers=pair,
                interval='1m',
                start=start_date.strftime('%Y-%m-%d'),
                end=end_date.strftime('%Y-%m-%d'),
                progress=False
            )
            
            # Check if data was retrieved
            if data.empty:
                print(f"No data retrieved for {pair}. Check if the ticker symbol is correct.")
                continue
                
            # Add pair identifier
            data['Ticker'] = pair
            
            # Save to CSV
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{pair.replace('=', '_')}_1m_{timestamp}.csv"
            filepath = os.path.join(output_dir, filename)
            data.to_csv(filepath)
            
            print(f"Saved {len(data)} records for {pair} to {filepath}")
            
            # Store in dictionary
            all_data[pair] = data
            
            # Avoid rate limiting
            time.sleep(1)
            
        except Exception as e:
            print(f"Error downloading {pair}: {str(e)}")
    
    return all_data

# Common currency pairs with their Yahoo Finance ticker symbols
COMMON_CURRENCY_PAIRS = [
    'EURUSD=X',  # Euro to US Dollar
    'GBPUSD=X',  # British Pound to US Dollar
    'USDJPY=X',  # US Dollar to Japanese Yen
    'AUDUSD=X',  # Australian Dollar to US Dollar
    'USDCAD=X',  # US Dollar to Canadian Dollar
    'USDCHF=X',  # US Dollar to Swiss Franc
    'NZDUSD=X',  # New Zealand Dollar to US Dollar
    'EURJPY=X',  # Euro to Japanese Yen
    'GBPJPY=X',  # British Pound to Japanese Yen
    'EURGBP=X',  # Euro to British Pound
]

if __name__ == "__main__":
    # You can define your own list of currency pairs or use the common ones
    # Uncomment and modify the line below to define your own pairs
    # my_pairs = ['EURUSD=X', 'GBPUSD=X', 'USDJPY=X']
    
    # Or use the predefined common pairs
    my_pairs = COMMON_CURRENCY_PAIRS
    
    # Download 1-minute data (maximum 7 days available)
    currency_data = download_1min_currency_data(my_pairs)
    
    print("\nData download complete!")
    
    # Optional: Print sample of the data
    for pair, data in currency_data.items():
        if not data.empty:
            print(f"\nSample data for {pair}:")
            print(data.head(3))

Saved 6974 records for EURUSD=X to currency_data/EURUSD_X_1m_20250327_113709.csv
Saved 6974 records for GBPUSD=X to currency_data/GBPUSD_X_1m_20250327_113710.csv
Saved 7035 records for USDJPY=X to currency_data/USDJPY_X_1m_20250327_113711.csv
Saved 6833 records for AUDUSD=X to currency_data/AUDUSD_X_1m_20250327_113712.csv
Saved 7034 records for USDCAD=X to currency_data/USDCAD_X_1m_20250327_113713.csv
Saved 6975 records for USDCHF=X to currency_data/USDCHF_X_1m_20250327_113715.csv
Saved 3516 records for NZDUSD=X to currency_data/NZDUSD_X_1m_20250327_113716.csv
Saved 6991 records for EURJPY=X to currency_data/EURJPY_X_1m_20250327_113717.csv
Saved 6995 records for GBPJPY=X to currency_data/GBPJPY_X_1m_20250327_113718.csv
Saved 6991 records for EURGBP=X to currency_data/EURGBP_X_1m_20250327_113720.csv

Data download complete!

Sample data for EURUSD=X:
Price                         Close      High       Low      Open   Volume  \
Ticker                     EURUSD=X  EURUSD=X  EURUSD=X  EUR